In [2]:
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score 

torch.manual_seed(1234)

n = 500
x = torch.rand(n, 2)
y = torch.Tensor([[1] if (k[0]+k[1]<1 or k[1]-k[0]<0)  else [0] for k in x])

class mm_cls(torch.nn.Module):
    def __init__(self):
        super(mm_cls, self).__init__()
        self.l1 = torch.nn.Linear(2, 5)
        self.l2 = lambda x: x**2
        self.l3 = torch.nn.Linear(5, 2)
        self.l4 = nn.Softmax(dim=1)
        
    def forward(self, x):
        x1 = self.l1(x)
        x2 = self.l2(x1)
        x3 = self.l3(x2)
        x4 = self.l4(x3)
        return x4
    
mm = mm_cls()

loss_fun = torch.nn.MSELoss()
optimizer = torch.optim.Adam(mm.parameters(),lr=0.001)

epochs = 2000
for i in range(epochs):
    y_hat = mm.forward(x)

    loss = loss_fun(y_hat, y)
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print('step = %4d'%i, 'loss = %.4e'%loss.item())

    optimizer.zero_grad

y_hat = y_hat[:,0] < y_hat[:,1]
y = y[:,0] < y[:,1]

acc = accuracy_score(y_hat, y)
print('acc = %f' % acc)

import matplotlib.pyplot as plt

plt.subplot(121)
plt.plot(x[y==1, 0], x[y==1, 1], 'om')
plt.plot(x[y==0, 0], x[y==0, 1], 'ob')
plt.title('original')

plt.subplot(122)
plt.plot(x[y_hat==1, 0], x[y_hat==1, 1], 'om')
plt.plot(x[y_hat==0, 0], x[y_hat==0, 1], 'ob')
plt.title('predicted')


step =    0 loss = 2.6644e-01
step =  100 loss = 2.5338e-01
step =  200 loss = 2.5284e-01
step =  300 loss = 2.5004e-01
step =  400 loss = 2.5242e-01
step =  500 loss = 2.5332e-01
step =  600 loss = 2.5238e-01
step =  700 loss = 2.5238e-01
step =  800 loss = 2.5256e-01
step =  900 loss = 2.5036e-01
step = 1000 loss = 2.5245e-01
step = 1100 loss = 2.5185e-01
step = 1200 loss = 2.5520e-01
step = 1300 loss = 2.6737e-01
step = 1400 loss = 2.6596e-01
step = 1500 loss = 2.7246e-01
step = 1600 loss = 2.6005e-01
step = 1700 loss = 2.6196e-01
step = 1800 loss = 2.5313e-01
step = 1900 loss = 2.6177e-01


IndexError: index 1 is out of bounds for dimension 1 with size 1

In [24]:
import torch
from torch.autograd import Variable

# 一定要继承 nn.Module
class TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TwoLayerNet, self).__init__()
        self.twolayernet = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        y_pred = self.twolayernet(x)
        return y_pred

# M是样本数量，input_size是输入层大小， hidden_size是隐含层大小，output_size是输出层大小
M, input_size, hidden_size, output_size = 64, 2, 2, 1
n = 100
# 生成随机数当作样本
x = torch.rand(n, 2)
y = torch.Tensor([[1] if (k[0]+k[1]<1 or k[1]-k[0]<0)  else [0] for k in x])

model = TwoLayerNet(input_size, hidden_size, output_size)

# 定义损失函数
loss_fn = nn.MSELoss(size_average=False)

learning_rate = 1e-4
EPOCH = 300

# 使用optim包来定义优化算法
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(EPOCH):    
    y_pred= model(x)
    loss = loss_fn(y_pred, y)
    if (t+1) % 50 == 0:
        print(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


tensor(21.7160, grad_fn=<MseLossBackward>)
tensor(19.6735, grad_fn=<MseLossBackward>)
tensor(19.0796, grad_fn=<MseLossBackward>)
tensor(18.5621, grad_fn=<MseLossBackward>)
tensor(18.0905, grad_fn=<MseLossBackward>)
tensor(17.6884, grad_fn=<MseLossBackward>)
/Users/zed/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [10]:
acc = accuracy_score(y_pred, y)
print('acc = %f' % acc)

ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [61]:
# 利用Pytorch解决XOR问题
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
n = 500
x = torch.rand(n, 2)
y = torch.Tensor([ 1 if (k[0]+k[1]<1 or k[1]-k[0]<0)  else 0 for k in x])



# 初始化权重变量
def weight_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 1.)
        m.bias.data.fill_(0.)


class XOR(nn.Module):
    def __init__(self):
        super(XOR, self).__init__()
        self.fc1 = nn.Linear(2, 2)   # 一个隐藏层 2个神经元
        self.fc2 = nn.Linear(2,2)   # 输出层 1个神经元
        self.fc3 = nn.Linear(2,1)
    def forward(self, x):
        h1 = F.sigmoid(self.fc1(x))  # 之前也尝试过用ReLU作为激活函数, 太容易死亡ReLU了.
        h2 = F.sigmoid(self.fc2(h1))
        h3 = torch.sign(self.fc3(h2))
        return h3


net = XOR()
net.apply(weight_init_normal)
x = torch.Tensor(x.reshape(-1, 2))
y = torch.Tensor(y.reshape(-1, 1))

# 定义loss function
criterion = nn.BCELoss()  # MSE
# 定义优化器
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)  # SGD
# 训练
for epoch in range(100000):
    optimizer.zero_grad()   # 清零梯度缓存区
    out = net(x)
    loss = criterion(out, y)
    loss.backward()
    optimizer.step()  # 更新

# 测试
test = net(x)
print()



/Users/zed/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



In [56]:
sum(out.detach() == y.detach())/y.detach().shape[0]

tensor([0.7700])